<a href="https://colab.research.google.com/github/cheffjiu/pytorch-tutorials-zh/blob/main/optimization_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\| **Optimization** \|\| [Save &
Load Model](saveloadrun_tutorial.html)

Optimizing Model Parameters(优化模型参数)
===========================

既然我们已经有了模型和数据，那么就该通过在数据上优化模型参数来对模型进行训练、验证和测试了。训练模型是一个迭代的过程；在每次迭代中，模型会对输出进行预测，计算预测中的误差（*损失*），收集误差相对于其参数的导数（正如我们在 [上一节](autograd_tutorial.html) 中看到的），然后使用梯度下降法 **优化（optimizes）** 这些参数。想要更详细地了解这个过程，可以查看 [3Blue1Brown 关于反向传播的这个视频](https://www.youtube.com/watch?v=tIeHLnjs5U8)。

Prerequisite Code
-----------------

我们加载前几节中关于[数据集与数据加载器](data_tutorial.html)以及[构建模型](buildmodel_tutorial.html)的代码。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

Hyperparameters(超参数)
===============

超参数是可调节的参数，可让您控制模型优化过程。不同的超参数值会影响模型训练和收敛速度（[详细了解](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)超参数调优）

我们为训练定义了以下超参数：

   -   **Number of Epochs** - 对数据集进行迭代的次数
   -   **Batch Size** - 在参数更新之前通过网络传播的数据样本数量
   -   **Learning Rate** -每次批次/轮次中模型参数更新的幅度。值越小，学习速度越慢；而值越大，在训练过程中可能会导致不可预测的情况。

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

Optimization Loop（优化循环）
=================

一旦我们设置好超参数，就可以通过优化循环来训练和优化模型。优化循环的每次迭代都被称为一个**epoch**.

每个时期都由两个主要部分组成：

  -   **The Train Loop** - 遍历训练数据集，并尝试收敛到最优参数。
  -   **The Validation/Test Loop** - 遍历测试数据集，检查模型性能是否在提升。

让我们简要熟悉一下训练循环中使用的一些概念。直接跳到优化循环的 `full-impl-label` 查看。

Loss Function（损失函数）
-------------

当提供一些训练数据时，我们未经训练的网络很可能无法给出正确答案。**损失函数（Loss function）**衡量所获得结果与目标值的差异程度，而我们在训练过程中希望最小化的正是损失函数。为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实数据标签值进行比较。

常见的损失函数包括
[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)
(Mean Square Error) 用于回归任务, 和
[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)
(Negative Log Likelihood) 用于分类.
[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)
结合了`nn.LogSoftmax`和`nn.NLLLoss`。

我们将模型的输出logits传递给`nn.CrossEntropyLoss`，它将对logits
进行归一化并计算预测误差。


In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

Optimizer(优化器)
=========

优化是在每个训练步骤中调整模型参数以减少模型误差的过程。 **优化算法（Optimization algorithms）** 定义
此过程是如何执行的（在此示例中，我们使用随机梯度下降法）。所有优化逻辑都封装在 `optimizer` 对象中。在这里，我们使用随机梯度下降（SGD）优化器；此外，PyTorch 中有许多[不同的优化器](https://pytorch.org/docs/stable/optim.html)，如 ADAM 和 RMSProp，它们对不同类型的模型和数据效果更好。

我们通过注册需要训练的模型参数并传入学习率超参数来初始化优化器。

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中，优化过程分为三个步骤：
  -   调用`optimizer.zero_grad()`重置模型参数的梯度。默认情况下，梯度会累加；为避免重复计算，我们在每次迭代时显式地将梯度清零。
  -   通过调用`loss.backward()`反向传播预测损失。PyTorch 会计算损失相对于每个参数的梯度。
  -   一旦我们得到了梯度，就调用`optimizer.step()`，根据反向传播中收集的梯度来调整参数。

Full Implementation {full-impl-label}（完整实现）
===================

我们定义`train_loop`，它循环执行优化代码，还定义`test_loop`，用于根据测试数据评估模型的性能。


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们初始化损失函数和优化器，并将其传递给`train_loop`和`test_loop`。你可以随意增加训练轮数，以跟踪模型不断提升的性能。


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Further Reading
===============

-   [Loss
    Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
-   [torch.optim](https://pytorch.org/docs/stable/optim.html)
-   [Warmstart Training a
    Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)
